In [1]:
import pyautogui as pag
import numpy as np
import mss, cv2, win32gui, win32ui, win32con, pdb, time, mouse
import matplotlib.pyplot as plt
import copy
import multiprocessing as mp

In [11]:
pos = pag.position()
print(pos)

Point(x=74, y=831)


In [2]:
def check_num(color):
    global color_list
    color_list = np.array([[255, 246, 222], # 0 
                            [255, 254, 255], # 2
                            [255, 236, 170], # 4
                            [255, 229, 143], # 8
                            [255, 216,  87], # 16
                            [241, 183,   0], # 32
                            [204, 140,   1], # 64
                            [ 60, 240, 141], # 128
                            [ 32, 232, 109], # 256z
                            [ 26, 222,  77], # 512
                            [121, 225, 255], # 1024
                            [ 13, 201, 255], # 2048 
                            [  1, 151, 255]]) # 4096
#     color_list2 = np.array([[236, 226, 203], # 0
#                             [235, 235, 235], # 2
#                             [237, 215, 156], # 4
#                             [236, 207, 131], # 8
#                             [237, 193,  80], # 16
#                             [226, 161,   1], # 32
#                             [209, 149,   2] # 64
#                             [ 60, 205, 131], # 128
#                             # 256
#                             [ 32, 185,  73], # 512
#                             [104, 191, 220]# 1024
#                             [ 19, 184, 241]]) # 2048
# [[[ 61 205 131]]]
# [[[ 32 185  73]]]
# [[[ 19 184 241]]]
# [[[  9 147 244]]]
# [[[235 235 235]]]
# [[[226 161   1]]]
# [[[235 235 235]]]
# [[[236 207 131]]]
# [[[237 215 156]]]
# [[[236 207 131]]]
# [[[237 193  80]]]
# [[[235 235 235]]]
# [[[236 226 203]]]
# [[[235 235 235]]]
# [[[237 215 156]]]
# [[[235 235 235]]]
# [ 60 205 131]]]
# [ 32 185  73]]]
# [ 19 184 241]]]
# [  9 147 244]]]

# [235 235 235]]]
# [226 161   1]]]
# [235 235 235]]]
# [236 207 131]]]

# [237 215 156]]]
# [236 207 131]]]
# [237 193  80]]]
# [235 235 235]]]

# [236 226 203]]]
# [236 235 235]]]
# [237 215 156]]]
# [235 235 235]]]
    color = color.flatten()
    loc = np.argmin(np.sum((color_list - color)**2, axis=1))
    return loc.item()

def get_tile_array(tile_xpos, tile_ypos):
    tile_array = np.zeros((4, 4))
    for ii in range(4):
        for jj in range(4):
            with mss.mss() as sct:
                color = np.array(sct.grab(
                    {'left':tile_xpos[ii], 'top':tile_ypos[jj], 
                     'width':1, 'height':1}))[:, :, :3]
                pass
            # print(color)
            tile_array[jj, ii] = check_num(color)
    return tile_array

def after_moving(tile_array, direction='up'):
    if direction == 'up':
        tile_array0 = copy.deepcopy(tile_array)
    if direction == 'down':
        tile_array0 = copy.deepcopy(tile_array[::-1])
    if direction == 'right':
        tile_array0 = copy.deepcopy(tile_array.T[::-1])
    if direction == 'left':
        tile_array0 = copy.deepcopy(tile_array.T)

    templet = np.zeros((4, 4))
    for i in range(4): # summing
        part = tile_array0[:, i]
        part0 = part[part > 0]
        for j in range(len(part0)-1):
            if part0[j] == part0[j+1]:
                part0[j] += 1
                part0[j+1] = 0
        part1 = part0[part0 > 0]
        templet[:len(part1), i] = part1

    if direction == 'down':
        templet = templet[::-1]
    if direction == 'right':
        templet = templet[::-1].T
    if direction == 'left':
        templet = templet.T
    return templet

def evaluation(after_tile_array, table):
    # print(table)
    dum = after_tile_array
    # dum = np.where(after_tile_array==0, 2, after_tile_array)
    return np.sum(table*(2**dum))

def find_direction(tile_array, depth=3):
    dir_order = ['left', 'up', 'right', 'down']
    direct0 = []
    for dd in dir_order:
        direct0.append(after_moving(tile_array, direction=dd))
    depth0 = [depth]*4
    if __name__ == "__main__":
        with mp.Pool(processes=4) as pool:
            results = np.asarray(pool.map(find_direct, list(zip(direct0, depth0, strict=True))))
    return results
    # results = []
    # with mp.Pool(processes=4) as pool:
    #     for ii, try_dir in enumerate(dir_order):
    #         after = after_moving(tile_array, direction=try_dir)
    #         results.append(pool.apply_async(find_direct, (after, depth-1)))
    # results = [res.get() for res in results]
    # results = np.array(results)
    # return dir_order[np.argmax(results)], np.max(results)
    
def find_direct(tile_array, depth=3):
    import numpy as np
    if np.sum(tile_array >= 9) > 3: return 'up', -1
    if (depth % 2) == 1:
        dir_order = ['left', 'up', 'right', 'down']
        score = np.zeros(len(dir_order))
        for ii, try_dir in enumerate(dir_order):
            after = after_moving(tile_array, direction=try_dir)
            if (after == tile_array).all(): 
                score[ii] = -depth
            elif depth == 1:
                score[ii] = evaluation(after, table)
            else:
                score[ii] = find_direct(after, depth=depth-1)
        return dir_order[np.argmax(score)], np.max(score)    
    else : #depth // 2 == 0:
        empty = np.where(tile_array == 0)
        dir_score = []
        for ii in range(len(empty[0])):
            tile_array[empty[0][ii], empty[1][ii]] = 1
            direct_sub, score_sub = find_direct(tile_array, depth=depth-1)
            dir_score.append(score_sub)
            # for jj in range(2):
            #     tile_array[empty[0][ii], empty[1][ii]] = jj + 1
            #     direct_sub, score_sub = find_direct(tile_array, depth=depth-1)
            #     fact = 0.9 if jj == 0 else 0.1
            #     dir_score.append(score_sub*fact)
        return np.mean(dir_score)
    
def drag(direction):
    delta = 30
    x, y = 220, 779
    if direction=='up': mouse.drag(x, y, x, y-delta)
    if direction=='down': mouse.drag(x, y, x, y+delta)    
    if direction=='right': mouse.drag(x, y, x+delta, y)
    if direction=='left': mouse.drag(x, y, x-delta, y)


In [3]:
challenge_mode_pos = [220, 719]
practice_mode_pos = [220, 779]
tile_xpos = np.arange(4)*100 + 50
tile_ypos = np.arange(4, dtype=int)*100 + 550

global pool
pool = mp.Pool(processes=4)
global table
table0 = np.arange(4*4)[::-1].reshape(4, 4)
table = 4.**(table0)
# table = table0**2
table[1] = table[1][::-1]
table[3] = table[3][::-1]
# table[0, 0] *= 2
# table[0, 0] = table[0, 0]*1e4
# table = table * 10**np.arange(4)[::-1, None]

# table = np.array([[10,8,7,6.5],
#  [.5,.7,1,3],
#  [-.5,-1.5,-1.8,-2],
#  [-3.8,-3.7,-3.5,-3]])

# aa = 2**np.arange(4)[::-1]
# table = aa*aa[:, None]

In [ ]:
tile_array = np.array([[3, 2, 2, 4], 
                       [1, 2, 2, 0], 
                       [0, 0, 0, 0], 
                       [1, 1, 0, 0]])
if __name__ == "__main__":
    final_dir, score = find_direction(tile_array)
    final_dir, score

In [18]:
mode = 'challenge'
mode = 'practice'
start_pos = challenge_mode_pos if mode =='challenge' else practice_mode_pos
pag.click(x = start_pos[0], y = start_pos[1])
time.sleep(3)
err = 0

history = []
next = 0
while True:
    if next :
        # print(num)
        num = 1
        while True:
            tile_array = np.array(get_tile_array(tile_xpos, tile_ypos))
            filled  = expect != 0
            if (expect[filled] == tile_array[filled]).all() or \
                num == 10 or \
                np.sum(filled) < 10 or \
                np.max(expect) < 9: 
                # if (num != 1) and (num != 10): print(num)
                break
            else:
                num += 1
                time.sleep(0.005)
    tile_array = get_tile_array(tile_xpos, tile_ypos)
    history.append(tile_array)
    final_dir, score = find_direct(tile_array, depth = 3)
    err = err + 1 if score == -1 else 0
    if err == 8: break
    drag(final_dir)
    history.append(final_dir)
    next = 1
    expect = np.array(after_moving(tile_array, final_dir))
    time.sleep(0.08)


In [5]:
history

[array([[0., 0., 1., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]),
 'left',
 array([[0., 0., 1., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]),
 'left',
 array([[1., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]),
 'right',
 array([[1., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 2.]]),
 'up',
 array([[0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 2.]]),
 'up',
 array([[0., 1., 0., 2.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]),
 'left',
 array([[0., 2., 0., 2.],
        [0., 1., 0., 1.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]),
 'left',
 array([[3., 0., 0., 0.],
        [2., 1., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]),
 'up',
 array([[3., 0., 0., 0.],
        [2., 1., 0., 0.],
        [0., 0., 0., 0.],
       

In [38]:
prepre = np.array([[3., 1., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

pre = np.array(after_moving(prepre, 'left'))
aft = np.array([[3., 1., 1., 1.],
        [1., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

filled = pre != 0
if (pre[filled] != aft[filled]).any():
    print(ii)
    print(pre)
    print(aft)

554
[[3. 2. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[3. 1. 1. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [36]:
pre[filled]

array([3., 2., 1.])

In [37]:
aft[filled]

array([3., 1., 1.])

In [30]:
for ii in range(len(history)//2-1):
    pre = np.array(after_moving(history[2*ii], history[2*ii+1]))
    aft = np.array(history[2*ii + 2])
    filled = pre != 0
    if (pre[filled] != aft[filled]).all():
        print(ii)
        print(pre)
        print(aft)

548
[[10.  2.  2.  2.]
 [ 0. 10.  6. 10.]
 [ 0.  0.  9.  0.]
 [ 0.  0.  0.  0.]]
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [9. 9. 6. 9.]
 [9. 9. 9. 9.]]
549
[[ 2.  2.  2.  2.]
 [10. 10.  6. 10.]
 [ 0.  0.  9.  0.]
 [ 0.  0.  0.  0.]]
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [9. 9. 6. 9.]
 [9. 9. 9. 9.]]
550
[[ 2.  2.  2.  2.]
 [10. 10.  6. 10.]
 [ 0.  0.  9.  0.]
 [ 0.  0.  0.  0.]]
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [9. 9. 6. 9.]
 [9. 9. 9. 9.]]
551
[[ 2.  2.  2.  2.]
 [10. 10.  6. 10.]
 [ 0.  0.  9.  0.]
 [ 0.  0.  0.  0.]]
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [9. 9. 6. 9.]
 [9. 9. 9. 9.]]
552
[[ 2.  2.  2.  2.]
 [10. 10.  6. 10.]
 [ 0.  0.  9.  0.]
 [ 0.  0.  0.  0.]]
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [9. 9. 6. 9.]
 [9. 9. 9. 9.]]
553
[[ 2.  2.  2.  2.]
 [10. 10.  6. 10.]
 [ 0.  0.  9.  0.]
 [ 0.  0.  0.  0.]]
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [9. 9. 6. 9.]
 [9. 9. 9. 9.]]
554
[[ 2.  2.  2.  2.]
 [10. 10.  6. 10.]
 [ 0.  0.  9.  0.]
 [ 0.  0.  0.  0.]]
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [9. 9. 6. 9.]
 [9. 9. 9. 9.]]


In [ ]:
while True:
    tile_array = get_tile_array(tile_xpos, tile_ypos)
    final_dir, score = find_direct(tile_array, 0)
    print("\r", final_dir, score, end="")
    # time.sleep(0)

In [29]:
5 // 2

2

In [9]:
tile_array = get_tile_array(tile_xpos, tile_ypos)
tile_array

[[[255 230 141]]]
[[[255 235 170]]]
[[[255 251 234]]]
[[[255 246 222]]]
[[[ 61 241 139]]]
[[[255 255 255]]]
[[[253 230 141]]]
[[[255 246 221]]]
[[[240 184   0]]]
[[[255 228 139]]]
[[[255 246 221]]]
[[[255 246 221]]]
[[[255 216  86]]]
[[[255 254 255]]]
[[[255 246 221]]]
[[[255 246 221]]]


array([[3., 7., 5., 4.],
       [2., 1., 3., 1.],
       [0., 3., 0., 0.],
       [0., 0., 0., 0.]])

In [20]:
tile_array = get_tile_array(tile_xpos, tile_ypos)
tile_array

array([[0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [58]:
aa = np.array([[0., 0., 2., 1.],
               [0., 0., 0., 0.],
               [0., 0., 0., 0.],
               [0., 0., 0., 0.]])
bb = np.array([[0., 0., 1., 1.],
               [0., 0., 0., 0.],
               [0., 0., 0., 1.],
               [0., 0., 0., 0.]])
filled = aa != 0
if (aa[filled] ==  bb[filled]).all(): 
    print('a')
else:
    print('b')

# np.logical_or(blank, new)

b


In [47]:
bb[filled]

array([1., 1.])

In [71]:
tile_array = np.array([[3, 2, 2, 4], 
                       [1, 2, 2, 0], 
                       [0, 0, 0, 0], 
                       [1, 1, 0, 0]])
# tile_array = get_tile_array(tile_xpos, tile_ypos)
final_dir, score = find_direct(tile_array)   
final_dir, score

('left', np.float64(-1.0))

In [68]:
pdb.pm()

> c:\users\khcho\miniforge3\lib\site-packages\multiprocess\pool.py(774)get()
    772             return self._value
    773         else:
--> 774             raise self._value
    775 
    776     def _set(self, i, obj):



ipdb>  u


> c:\users\khcho\appdata\local\temp\ipykernel_17728\2442304970.py(149)find_direction()



ipdb>  results


[<multiprocess.pool.ApplyResult object at 0x00000145452F1940>, <multiprocess.pool.ApplyResult object at 0x00000145449862D0>, <multiprocess.pool.ApplyResult object at 0x0000014544985A30>, <multiprocess.pool.ApplyResult object at 0x0000014544986270>]


...

ipdb>  results[0].get()


*** NameError: name 'np' is not defined


ipdb>  import numpy as np
ipdb>  results[0].get()


*** NameError: name 'np' is not defined


ipdb>  q


In [73]:
get_tile_array(tile_xpos, tile_ypos)

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [67]:
tile_array == after_moving(tile_array, 'left')

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])